In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [10]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
y.shape, tX.shape

((250000,), (250000, 30))

In [67]:
tX

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

## Do your thing crazy machine learning thing here :) ...

In [94]:
w_initial = np.zeros(tX.shape[1])
#[weights, loss]=least_squares_GD(y, tX, w_initial, 10, 0.000001)
#[weights, loss]= least_squares(y, tX)
[weights, loss] = least_squares_SGD(y, tX, w_initial, 1000, 0.00000001)
weights, loss

(array([ 3.90247269e-04, -1.01496705e-04, -2.10177574e-05,  4.62920587e-05,
        -8.70520860e-06,  2.72639183e-04, -1.29289731e-05, -5.39026732e-08,
        -1.20059788e-05, -4.12259426e-06, -1.68796735e-06,  1.75711574e-06,
        -9.55400505e-06,  3.02542334e-05, -2.31302958e-07,  1.22834608e-08,
        -1.61040003e-05, -2.52687839e-07,  8.58454619e-07, -1.08486154e-05,
         1.23093084e-07, -2.05174172e-05, -6.24290438e-07,  1.04865452e-05,
        -5.59317285e-06, -4.96168058e-06, -2.50828966e-05, -9.80608251e-06,
        -9.80138836e-06, -1.82727998e-05]), 0.42759158179949736)

## Generate predictions and save ouput in csv format for submission:

In [53]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [56]:
OUTPUT_PATH = '../data/pred.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)